In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

2024-03-24 17:08:06.984360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Reading Data
with open(r'/Volumes/Projects/Word_prediction_LSTM/dataset/IndiaUS.txt', 'r', encoding='utf-8') as f:
    mytext = f.read()

In [12]:
print(mytext)

Following a lavish state visit by Indian Prime Minister Narendra Modi to Washington, US President Joe Biden has called his country's partnership with India among the "most consequential in the world". The BBC's Vikas Pandey and Soutik Biswas explore the factors that contribute to the visit's potential in strengthening the ties between the two nations.

The US's relationship with India - the world's most populous country - is "stronger, closer and more dynamic than any time in history", Mr Biden said at the completion of a pomp-filled state visit by Mr Modi to the White House.

The remark may not be an exaggeration. "This summit suggests that the relationship has been transformed. It underscores just how broad and deep it has become in a relatively short time," says Michael Kugelman of The Wilson Center, an American think-tank.

A key reason is that Washington is keen to draw India closer so that it can act as a counterbalance to China's growing influence in the Indo-Pacific. India-US t

In [15]:
# Tokenize the dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts([mytext])
total_words = len(tokenizer.word_index)+1
print(total_words)

599


In [17]:
print(tokenizer.word_index)

{'the': 1, 'to': 2, 'in': 3, 'a': 4, 'and': 5, 'india': 6, 'of': 7, 'mr': 8, 'us': 9, 'is': 10, 'modi': 11, 'has': 12, 'that': 13, 'biden': 14, 'as': 15, 'with': 16, 'not': 17, "india's": 18, 'for': 19, 'but': 20, 'on': 21, 'washington': 22, 'an': 23, 'it': 24, 'says': 25, 'will': 26, 'are': 27, 'indian': 28, 'prime': 29, 'minister': 30, 'this': 31, 'have': 32, 'trade': 33, 'visit': 34, 'by': 35, 'president': 36, 'relationship': 37, 'been': 38, 'about': 39, 'strategic': 40, 'more': 41, 'up': 42, 'during': 43, 'also': 44, 'make': 45, 'sirohi': 46, 'china': 47, 'air': 48, 'force': 49, 'state': 50, 'his': 51, 'ties': 52, 'two': 53, 'at': 54, 'white': 55, 'house': 56, 'be': 57, 'lot': 58, 'ms': 59, 'technology': 60, 'jet': 61, 'semiconductor': 62, 'now': 63, 'first': 64, 'was': 65, 'global': 66, 'one': 67, 'he': 68, 'without': 69, 'they': 70, 'narendra': 71, 'joe': 72, 'world': 73, 'potential': 74, 'how': 75, 'become': 76, 'kugelman': 77, 'years': 78, 'there': 79, "modi's": 80, 'fighter': 

In [48]:
input_sequences = []
for line in mytext.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    # print(token_list)
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        # print(n_gram_sequence)
        
        input_sequences.append(n_gram_sequence)

In [49]:
input_sequences[:10]

[[99, 4],
 [99, 4, 177],
 [99, 4, 177, 50],
 [99, 4, 177, 50, 34],
 [99, 4, 177, 50, 34, 35],
 [99, 4, 177, 50, 34, 35, 28],
 [99, 4, 177, 50, 34, 35, 28, 29],
 [99, 4, 177, 50, 34, 35, 28, 29, 30],
 [99, 4, 177, 50, 34, 35, 28, 29, 30, 71],
 [99, 4, 177, 50, 34, 35, 28, 29, 30, 71, 11]]

In [50]:
max_sequence_len = max([len(seq) for seq in input_sequences])
print(max_sequence_len)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
print(input_sequences)

83
[[  0   0   0 ...   0  99   4]
 [  0   0   0 ...  99   4 177]
 [  0   0   0 ...   4 177  50]
 ...
 [  0   0   0 ... 176 598  25]
 [  0   0   0 ... 598  25  59]
 [  0   0   0 ...  25  59  46]]


In [65]:
input_sequences.shape

(1371, 83)

In [51]:
print(input_sequences[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0 99  4]


In [53]:
x = input_sequences[:,:-1]
y = input_sequences[:,-1]

In [54]:
print(x[0])
print(y[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0 99]
4


In [63]:
y

array([  4, 177,  50, ...,  25,  59,  46], dtype=int32)

In [59]:
len(x)

1371

In [62]:
len(y)

1371

In [64]:
# Convert y into one-hot encoding
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Model Training

In [69]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 82, 100)           59900     
                                                                 
 lstm_3 (LSTM)               (None, 150)               150600    
                                                                 
 dense_3 (Dense)             (None, 599)               90449     
                                                                 
Total params: 300949 (1.15 MB)
Trainable params: 300949 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
43/43 [==============================] - 7s 90ms/step - loss: 6.1613 - accuracy: 0.0496
Epoch 2/100
43/43 [==============================] - 4s 91ms/step - loss: 5.7934 - accuracy: 0.0576
Epoch 3/100
43/43 [==============================] - 4s 86ms/step - loss: 5.7271 - accuracy: 0.0576
Epoch 4/100
43/43 [==============================] - 4s 82ms/step - loss: 5.6741 - accuracy: 0.0576
Epoch 5/100
43/43 [==============================] - 4s 92ms/step - loss: 5.6015 - accuracy: 0.0576
Epoch 6/100
43/43 [==============================] - 4s 92ms/step - loss: 5.5012 - accuracy: 0.0576
Epoch 7/100
43/43 [==============================] - 3s 75ms/step - loss: 5.3779 - accuracy: 0.0627
Epoch 8/100
43/43 [==============================] - 3s 80ms/step - loss: 5.2072 - accuracy: 0.0985
Epoch 9/100
43/43 [==============================] - 3s 76ms/step - loss: 5.0033 - accuracy: 0.1116
Epoch 10/100
43/43 [==============================] - 4s 87ms/step - loss: 4.7707 - accuracy: 0.1284

In [88]:
input_text = "india"
predict_next_words = 2

for i in range(predict_next_words):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    print(token_list)

    token_list = pad_sequences([token_list], maxlen= max_sequence_len-1, padding="pre")
    predicted = np.argmax(model.predict(token_list), axis=-1)
    print(predicted)

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
            
    input_text += " " + output_word

print(input_text)

[6]
1/1 [==============================] - 0s 20ms/step
[26]
[6, 26]
1/1 [==============================] - 0s 23ms/step
[290]
india will proceed
